In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer,util
import re
import pickle

In [2]:
# df= pd.read_csv(r'C:\Users\sarthakkaushik\OneDrive - KPMG\Desktop\D Drive\KPMG\Maruti - Hazard Recommendation\Kick- OFF\Data\Code\Python_code\MSIL_updated_hazard_1k.csv',encoding='ISO-8859-1')
df= pd.read_csv(r'..\Data\Updated_ObservationsHazardMaster.csv',encoding='ISO-8859-1')

df.head()


,Audit Id,Audit Category,Audit Type,Location,Auditor Id,Auditor Name,Auditor Dept,Auditee Id,Auditee Name,Auditee Dept,...,Severity_mean,Severity_median,Severity_mode,Exposure_mean,Exposure_median,Exposure_mode,Likelihood_mean,Likelihood_median,Likelihood_mode,clean_text
0,2223/IA/SFTY/2232,PLANNED,Internal Audit,ROHTAK,268453,MR. JITENDER .,SFTY-R,562009,MR. DEEPAK BABBER,EMUP-P,...,10,10,10,10,10,10,10,10,10,disbalanced ac outdoor unit
1,2223/WP/SFTY/2797,PLANNED,Work permit,ROHTAK,259942,MR. DEEPAK MALIK,SFTY-R,259667,MR. SUBHRANSHU SWAIN,CVLP-P,...,25,20,20,15,15,15,20,20,20,poor accessibility availability and improper s...
2,2223/SHD/SFTY/2617,PLANNED,Shutdown Daily Audit,ROHTAK,259993,MR. JATINDER DUBEY,SFTY-R,567299,MR. DARSHAN PRAVIN UKALKAR,CVLP-P,...,25,20,20,15,15,15,20,20,20,poor accessibility availability and improper s...
3,2223/WP/SFTY/1941,PLANNED,Work permit,ROHTAK,259993,MR. JATINDER DUBEY,SFTY-R,246360,MR. ZULQUARNAIN SABIR,CVLP-P,...,25,20,20,15,15,15,20,20,20,poor accessibility availability and improper s...
4,2223/IA/SFTY/2092,PLANNED,Internal Audit,ROHTAK,268453,MR. JITENDER .,SFTY-R,259667,MR. SUBHRANSHU SWAIN,CVLM-RM,...,25,20,20,15,15,15,20,20,20,poor accessibility availability and improper s...


In [3]:
df.shape

(1721, 67)

In [4]:
df.columns

Index(['Audit Id', 'Audit Category', 'Audit Type', 'Location', 'Auditor Id',
       'Auditor Name', 'Auditor Dept', 'Auditee Id', 'Auditee Name',
       'Auditee Dept', 'Start Date', 'End Date', 'Reference',
       'Associated Hazard', 'Nature of Work', 'Safe Flag', 'Contract Id',
       'Recommendations', 'Counter Measure', 'Target Start Date',
       'Target End Date', 'Actual Start Date', 'Actual End Date',
       'Action Status', 'Observation Status', 'Audit Status', 'After Severity',
       'After Exposure', 'After Likelihood', 'After Risk',
       'After Risk Category', 'HIRA Updated', 'Hierarchy of Control',
       'Severity', 'Exposure', 'Likelihood', 'Risk', 'Before Risk',
       'Location.1', 'Observation Details', 'Project_site', 'New_Location',
       'Location Category', 'New_Hazard_Label', 'HazardA', 'HazardB',
       'HazardC', 'Hazard_mapping', 'Hazard Category', 'Severity_Cal',
       'Exposure_Cal', 'Likelihood_Cal', 'Risk_Cal', 'Risk_Category',
       'Hazards', 'New

In [5]:
# Load data and Sentence Transformer model
data=df.copy()

data.shape

(1721, 67)

In [6]:
# Load data and Sentence Transformer model
data=df.copy()
# filter only Location_category='Project
data=data[data['Project_site']=='yes']

def preprocess(df):
    #set up punctuations we want to be replaced
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
    REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")
    tempArr = []
    for line in df:
        # remove puctuation
        tmpL = REPLACE_NO_SPACE.sub("", line.lower()) # convert all tweets to lower cases
        tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
        tempArr.append(tmpL)
    return tempArr


# cleaning the text
data['clean_text'] = preprocess(data['Observation Details'])
# data=data[['New_Location','Observation Details','Nature of Work','HazardA','HazardB','HazardC', 'Hazard Category','Location Category','Hazard_Labeling', 'Associated Hazard','Severity', 'Exposure', 'Likelihood', 'Risk',
#        'Risk_Category', 'clean_text']]
data.head()

,Audit Id,Audit Category,Audit Type,Location,Auditor Id,Auditor Name,Auditor Dept,Auditee Id,Auditee Name,Auditee Dept,...,Severity_mean,Severity_median,Severity_mode,Exposure_mean,Exposure_median,Exposure_mode,Likelihood_mean,Likelihood_median,Likelihood_mode,clean_text
1,2223/WP/SFTY/2797,PLANNED,Work permit,ROHTAK,259942,MR. DEEPAK MALIK,SFTY-R,259667,MR. SUBHRANSHU SWAIN,CVLP-P,...,25,20,20,15,15,15,20,20,20,access rough of fire extinguisher are block by...
2,2223/SHD/SFTY/2617,PLANNED,Shutdown Daily Audit,ROHTAK,259993,MR. JATINDER DUBEY,SFTY-R,567299,MR. DARSHAN PRAVIN UKALKAR,CVLP-P,...,25,20,20,15,15,15,20,20,20,access to fire extinguisher blocked
3,2223/WP/SFTY/1941,PLANNED,Work permit,ROHTAK,259993,MR. JATINDER DUBEY,SFTY-R,246360,MR. ZULQUARNAIN SABIR,CVLP-P,...,25,20,20,15,15,15,20,20,20,access to fire extinguisher was blocked
5,2223/WP/SFTY/3571,PLANNED,Work permit,ROHTAK,268453,MR. JITENDER .,SFTY-R,259667,MR. SUBHRANSHU SWAIN,CVLP-P,...,25,20,20,15,15,15,20,20,20,fire extinguisher access blocked with construc...
6,2223/WP/SFTY/3548,PLANNED,Work permit,ROHTAK,268453,MR. JITENDER .,SFTY-R,259667,MR. SUBHRANSHU SWAIN,CVLP-P,...,25,20,20,15,15,15,20,20,20,fire extinguisher access was blocked


In [7]:
data.shape

(804, 67)

In [8]:
all_intents = list(data['clean_text'])
all_intents[:5]

['access rough of fire extinguisher are block by sand',
 'access to fire extinguisher blocked',
 'access to fire extinguisher was blocked',
 'fire extinguisher access blocked with construction site material',
 'fire extinguisher access was blocked']

In [9]:
def embed(model, model_type, sentences):
    if model_type == 'use':
        embeddings = model(sentences)
    elif model_type == 'sentence transformer':
        embeddings = model.encode(sentences, show_progress_bar=True)
    
    return embeddings

In [10]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embed(model, 'sentence transformer', all_intents)


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

In [11]:
embeddings[:5]

array([[-0.00125374,  0.07001618,  0.00070843, ...,  0.02426864,
         0.03226119,  0.08377783],
       [-0.02571283,  0.06142708, -0.06532201, ..., -0.00902907,
         0.0086051 ,  0.07254763],
       [-0.00098837,  0.06524634, -0.05864475, ..., -0.01829678,
        -0.01031721,  0.06459586],
       [-0.0302797 ,  0.05683879, -0.01923598, ..., -0.06037818,
         0.04221794,  0.09569596],
       [ 0.01012955,  0.06736135, -0.05082543, ..., -0.02347978,
        -0.01805254,  0.07514797]], dtype=float32)

In [12]:
#Saving data as dictionary
# Convert the DataFrame to a dictionary
data_dict = data.to_dict('list')
data_dict['Obs_Embeddings']=embeddings
data['Obs_embd']=list(embeddings)
# data_dict

In [13]:
# Get all the keys in the dictionary
keys = data_dict.keys()

# Print the keys
print(keys)

dict_keys(['Audit Id', 'Audit Category', 'Audit Type', 'Location', 'Auditor Id', 'Auditor Name', 'Auditor Dept', 'Auditee Id', 'Auditee Name', 'Auditee Dept', 'Start Date', 'End Date', 'Reference', 'Associated Hazard', 'Nature of Work', 'Safe Flag', 'Contract Id', 'Recommendations', 'Counter Measure', 'Target Start Date', 'Target End Date', 'Actual Start Date', 'Actual End Date', 'Action Status', 'Observation Status', 'Audit Status', 'After Severity', 'After Exposure', 'After Likelihood', 'After Risk', 'After Risk Category', 'HIRA Updated', 'Hierarchy of Control', 'Severity', 'Exposure', 'Likelihood', 'Risk', 'Before Risk', 'Location.1', 'Observation Details', 'Project_site', 'New_Location', 'Location Category', 'New_Hazard_Label', 'HazardA', 'HazardB', 'HazardC', 'Hazard_mapping', 'Hazard Category', 'Severity_Cal', 'Exposure_Cal', 'Likelihood_Cal', 'Risk_Cal', 'Risk_Category', 'Hazards', 'New_label', 'Hazards_cluster', 'Severity_mean', 'Severity_median', 'Severity_mode', 'Exposure_mea

In [14]:
len(data_dict['Obs_Embeddings'])

804

In [15]:
# Save the datframe to a binary file using pickle
with open('..\Embeddings\embeddings_df.pkl', 'wb') as f:
    pickle.dump(data, f)


In [16]:
# Load the dictionary from the binary file using pickle
with open('..\Embeddings\embeddings_df.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

# Access the sentences and embeddings from the loaded dictionary
# loaded_sentences = loaded_data['sentences']
# loaded_embeddings = loaded_data['Obs_Embeddings']
obs_embd = loaded_data['Obs_embd']
obs_embd[1]

array([-1.25373527e-03,  7.00161755e-02,  7.08428794e-04, -2.75417697e-02,
        7.80179277e-02, -5.88851199e-02, -1.60048120e-02,  1.19803306e-02,
       -1.94527041e-02, -3.39021697e-03,  1.42624220e-02,  2.84954370e-03,
        2.37829033e-02,  4.01061997e-02,  2.01744605e-02,  2.97546964e-02,
       -3.61153483e-02,  3.91792832e-03,  1.52902892e-02, -1.87912919e-02,
        4.40936275e-02,  1.08584389e-01, -1.76141982e-03, -5.88597506e-02,
       -9.41022933e-02,  1.19627835e-02, -8.62515643e-02,  5.25557026e-02,
        6.36419058e-02, -3.39094400e-02,  3.29065472e-02, -6.58085421e-02,
       -6.12363704e-02,  3.25750746e-02,  6.40277863e-02, -9.61752012e-02,
       -6.13235421e-02,  1.15465082e-01, -2.33826675e-02,  7.43097216e-02,
       -6.60079494e-02,  2.80778296e-02,  3.96794751e-02, -6.62688315e-02,
        5.67321526e-03,  4.85005602e-03,  4.56781387e-02, -5.68684898e-02,
        4.07756194e-02, -7.35834055e-03,  2.11849511e-02,  1.66519880e-02,
       -1.15483388e-01, -

In [17]:
loaded_data

,Audit Id,Audit Category,Audit Type,Location,Auditor Id,Auditor Name,Auditor Dept,Auditee Id,Auditee Name,Auditee Dept,...,Severity_median,Severity_mode,Exposure_mean,Exposure_median,Exposure_mode,Likelihood_mean,Likelihood_median,Likelihood_mode,clean_text,Obs_embd
1,2223/WP/SFTY/2797,PLANNED,Work permit,ROHTAK,259942,MR. DEEPAK MALIK,SFTY-R,259667,MR. SUBHRANSHU SWAIN,CVLP-P,...,20,20,15,15,15,20,20,20,access rough of fire extinguisher are block by...,"[-0.0012537353, 0.070016176, 0.0007084288, -0...."
2,2223/SHD/SFTY/2617,PLANNED,Shutdown Daily Audit,ROHTAK,259993,MR. JATINDER DUBEY,SFTY-R,567299,MR. DARSHAN PRAVIN UKALKAR,CVLP-P,...,20,20,15,15,15,20,20,20,access to fire extinguisher blocked,"[-0.025712827, 0.06142708, -0.06532201, -0.019..."
3,2223/WP/SFTY/1941,PLANNED,Work permit,ROHTAK,259993,MR. JATINDER DUBEY,SFTY-R,246360,MR. ZULQUARNAIN SABIR,CVLP-P,...,20,20,15,15,15,20,20,20,access to fire extinguisher was blocked,"[-0.0009883732, 0.065246336, -0.058644753, 0.0..."
5,2223/WP/SFTY/3571,PLANNED,Work permit,ROHTAK,268453,MR. JITENDER .,SFTY-R,259667,MR. SUBHRANSHU SWAIN,CVLP-P,...,20,20,15,15,15,20,20,20,fire extinguisher access blocked with construc...,"[-0.030279702, 0.05683879, -0.019235978, 0.014..."
6,2223/WP/SFTY/3548,PLANNED,Work permit,ROHTAK,268453,MR. JITENDER .,SFTY-R,259667,MR. SUBHRANSHU SWAIN,CVLP-P,...,20,20,15,15,15,20,20,20,fire extinguisher access was blocked,"[0.010129548, 0.067361355, -0.050825432, 0.002..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,2223/WP/SFTY/2192,PLANNED,Work permit,ROHTAK,259942,MR. DEEPAK MALIK,SFTY-R,567299,MR. DARSHAN PRAVIN UKALKAR,CVLP-P,...,20,20,15,15,15,20,20,20,proper arrangement of scaffold not provided fo...,"[-0.03588171, -0.036095, 0.0026186896, 0.01429..."
1110,2223/DSP/SFTY/2682,PLANNED,Daily Safety Patrolling,ROHTAK,268453,MR. JITENDER .,SFTY-R,567299,MR. DARSHAN PRAVIN UKALKAR,CVLP-P,...,20,20,15,15,15,20,20,20,unsafe height work at column casting work adeq...,"[0.008001335, 0.037020788, -0.070061296, -0.07..."
1115,2223/WP/SFTY/2899,PLANNED,Work permit,ROHTAK,290955,MR. AMRIT SHAHI,SFTY-R,246360,MR. ZULQUARNAIN SABIR,CVLM-RM,...,20,20,15,15,15,20,20,20,damaged grinder wheel found at site which may ...,"[-0.10516074, -0.044714857, 0.010045687, 0.103..."
1116,2223/DSP/SFTY/1996,PLANNED,Daily Safety Patrolling,ROHTAK,290955,MR. AMRIT SHAHI,SFTY-R,249351,MR. GAUTAM JAIN,EN7F,...,20,20,15,15,15,20,20,20,damaged grinding wheel installed in grinding m...,"[-0.075917155, -0.025525656, 0.0029858376, 0.0..."
